<a href="https://colab.research.google.com/github/AmulPatil/mlops-zoomcamp/blob/main/MLOPS-Homework%20%231.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls cd  drive/MyDrive/temp/mlops

ls: cannot access 'cd': No such file or directory
drive/MyDrive/temp/mlops:
yellow_tripdata_2022-01.parquet  yellow_tripdata_2022-03.parquet
yellow_tripdata_2022-02.parquet  yellow_tripdata_2022-04.parquet


In [3]:
import pandas as pd
import numpy as np
import glob
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.suppress=True
import pandas as pd
pd.options.mode.chained_assignment = None

In [4]:
df_int = pd.read_parquet("drive/MyDrive/temp/mlops/yellow_tripdata_2022-01.parquet")

In [5]:
len(df_int.columns)#,df_int.info()

19

Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data
* :19

In [6]:

def preprocess(df,train=True):
  df['duration'] = df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']
  if train:
    print("Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip")
    print(df['duration'].std())
    print("Question 3: Fraction of the records left after dropping the outliers")
    print(100 - ((df.shape[0]-df[(df['duration']>pd.Timedelta(minutes=1))&(df['duration']<pd.Timedelta(minutes=60))].shape[0])/df.shape[0])*100)
  df = df[(df['duration']>pd.Timedelta(minutes=1))&(df['duration']<pd.Timedelta(minutes=60))]
  
  df['duration'] = df['duration'].dt.seconds/60
  df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype('str')
  return df

In [7]:
df = preprocess(df_int,train=True)

Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip
0 days 00:46:26.718308266
Question 3: Fraction of the records left after dropping the outliers
98.25883922885828


Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip
0 days 00:46:26.718308266 /n
Question 3: Fraction of the records left after dropping the outliers
98.25883922885828

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)? 515

In [8]:

vectorizer = DictVectorizer()
data_inter = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X = vectorizer.fit_transform(data_inter)


In [9]:
X.shape[1]

515

Question 4: Dimensionality after OHE? 515

In [10]:

y = df['duration']
reg = LinearRegression().fit(X, y)
reg.score(X, y)
print("Question 5: RMSE on train : "+str(mean_squared_error(y, reg.predict(X), squared=False)))


Question 5: RMSE on train : 6.983870768584342


In [11]:
df_val = pd.read_parquet('drive/MyDrive/temp/mlops/yellow_tripdata_2022-02.parquet')
df_val_process = preprocess(df_val,train=False)

In [12]:
data_inter_eval = df_val_process[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_eval = vectorizer.transform(data_inter_eval)
rms_eval = mean_squared_error(df_val_process['duration'], reg.predict(X_eval), squared=False)


In [13]:
print("Question 6: RMSE on validation: "+str(mean_squared_error(df_val_process['duration'], reg.predict(X_eval), squared=False)))


Question 6: RMSE on validation: 7.784425752834739
